In [80]:
import numpy as np

from data_loaders import *

# Instructions on how to load data

In [3]:
# How to read one file: careful, this might take a long time
# data is by defult QUANTILE scaled so between [0, 1]
a = read_scale(SAMPLE_PATH, no_obs=400000, slice_size=10)
a.shape

(399990, 200)

In [4]:
# How to read best bid and ask prices quickly
buy, sell = read_bid_ask(SAMPLE_PATH, no_obs=400000)
sell

0         3172.0
1         3172.8
2         3170.0
3         3174.4
4         3174.4
5         3172.8
6         3172.8
7         3172.2
8         3173.2
9         3172.6
10        3174.8
11        3175.0
12        3175.6
13        3176.0
14        3174.8
15        3175.2
16        3174.8
17        3175.0
18        3174.4
19        3174.0
20        3174.0
21        3174.2
22        3174.8
23        3174.0
24        3173.0
25        3174.0
26        3173.6
27        3174.0
28        3174.0
29        3173.0
           ...  
399970    2984.6
399971    2984.6
399972    2984.6
399973    2984.6
399974    2984.4
399975    2984.4
399976    2985.0
399977    2985.0
399978    2985.0
399979    2985.0
399980    2985.0
399981    2985.0
399982    2985.2
399983    2985.6
399984    2985.6
399985    2985.4
399986    2985.4
399987    2985.4
399988    2985.4
399989    2985.2
399990    2985.4
399991    2985.4
399992    2985.4
399993    2985.2
399994    2985.2
399995    2985.2
399996    2985.4
399997    2985

In [ ]:
# How to iterate over all datafiles available
#  the generator loops over all csv files in data/order_books
for a in generate_data(read_scale, no_obs=2000, slice_size=10):
    print(a)
    # Do anything you want with a, like train the model

# Sample model

In [13]:
# Let's fit a basic NN that tries to predict the best bid price
#  10 minutes in the future. Train it on first half of the SAMPLE_PATH file.
#  Report test performance on a different file.

offset = 120 # 10 minutes roughly
data_size = 200000 # size of train/test set
slice_size = 10

X = read_scale(SAMPLE_PATH, no_obs=2*data_size, slice_size=slice_size)
y = read_bid_ask(SAMPLE_PATH, no_obs=2*data_size+offset-slice_size)[1][offset:]

X_tr, X_ts = X[:data_size], X[data_size:]
y_tr, y_ts = y[:data_size], y[data_size:]

In [20]:
# Training model. Takes 25s on my laptop. 
# Ignore the ConvergenceWarning (need mroe data)

from sklearn.neural_network import MLPRegressor

nn = MLPRegressor(hidden_layer_sizes=(100, 100, 100,), max_iter=10)
nn.fit(X_tr, y_tr)

/Users/fabiopruneri/Documents/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100, 100, 100), learning_rate='constant',
             learning_rate_init=0.001, max_iter=10, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [21]:
nn.score(X_tr, y_tr) # This is the R^2
# As expected, performs very well on training data (massive overfitting)

0.8800552784956912

In [23]:
nn.score(X_ts, y_ts)
# And terribly on test data

-0.8562876471564211

In [97]:
df = pd.read_csv('data/order_books/2011/IF1101.csv').iloc[:, 1:].head(10)
price = (df['S1'] + df['B1']) / 2
print(df)
print(price)

       S5      S4      S3      S2      S1      B1      B2      B3      B4  \
0  3174.8  3174.6  3173.8  3173.0  3172.0  3171.8  3171.4  3171.0  3170.8   
1  3175.8  3175.0  3174.8  3173.8  3172.8  3172.2  3172.0  3171.8  3171.6   
2  3174.6  3174.4  3173.6  3171.2  3170.0  3169.8  3168.8  3168.0  3167.8   
3  3176.0  3175.8  3175.0  3174.6  3174.4  3173.2  3173.0  3172.8  3172.2   
4  3176.0  3175.8  3175.0  3174.6  3174.4  3172.8  3172.6  3172.2  3172.0   
5  3175.0  3174.6  3174.4  3173.4  3172.8  3172.0  3171.8  3171.0  3170.8   
6  3174.6  3174.4  3174.2  3173.4  3172.8  3172.4  3172.2  3172.0  3171.8   
7  3174.2  3173.4  3173.2  3172.8  3172.2  3172.0  3171.8  3171.0  3170.8   
8  3174.6  3174.4  3174.2  3173.4  3173.2  3172.6  3172.4  3172.2  3172.0   
9  3174.2  3173.8  3173.4  3173.0  3172.6  3172.4  3172.2  3172.0  3171.8   

       B5  SV5  SV4  SV3  SV2  SV1  BV1  BV2  BV3  BV4  BV5  
0  3170.4    1    2    3    1    5    1    3    2    6    1  
1  3171.4    2    2    1    

In [29]:
print(read(is_buy=True, path='data/order_books/2011/IF1101.csv', no_obs = 100))

        B1      B2      B3      B4      B5  BV1  BV2  BV3  BV4  BV5   price
0   3171.8  3171.4  3171.0  3170.8  3170.4    1    3    2    6    1  3171.9
1   3172.2  3172.0  3171.8  3171.6  3171.4    1   13    2    3    3  3172.5
2   3169.8  3168.8  3168.0  3167.8  3167.6    3    3    1    1   21  3169.9
3   3173.2  3173.0  3172.8  3172.2  3171.8    1    3    1    1    1  3173.8
4   3172.8  3172.6  3172.2  3172.0  3171.8    5    2    1    7    1  3173.6
5   3172.0  3171.8  3171.0  3170.8  3170.2    7    1   19    1   16  3172.4
6   3172.4  3172.2  3172.0  3171.8  3171.0    1    2    1    2   31  3172.6
7   3172.0  3171.8  3171.0  3170.8  3170.2    5    2   31    1    6  3172.1
8   3172.6  3172.4  3172.2  3172.0  3171.8    3    4    6    6    2  3172.9
9   3172.4  3172.2  3172.0  3171.8  3171.0    3    8   11    2   31  3172.5
10  3174.0  3173.2  3172.6  3172.2  3172.0    2    1    3    1   21  3174.4
11  3174.4  3172.6  3172.2  3172.0  3171.8    1    4    7   21    5  3174.7
12  3175.2  

In [95]:
def read(path, is_buy=False, no_obs=None, slice_size=50):
    df = pd.read_csv(path).iloc[:, 1:].head(no_obs)
    price = (df['S1'] + df['B1']) / 2
    df['price'] = price
    num_slices = int(no_obs / slice_size)
    if is_buy:
        new_df = pd.DataFrame(df, columns=['B1', 'B2', 'B3', 'B4', 'B5', 'BV1', 'BV2', 'BV3', 'BV4', 'BV5', 'price'])
    else:
        new_df = pd.DataFrame(df, columns=['S1', 'S2', 'S3', 'S4', 'S5', 'SV1', 'SV2', 'SV3', 'SV4', 'SV5', 'price'])
    return [new_df.iloc[n*slice_size:(n+1)*slice_size, :] for n in range(num_slices)]

In [96]:
for a in generate_data(read, 'data/order_books/2011/IF1101.csv', no_obs=10, slice_size=3):
    print(a)

[       B1      B2      B3      B4      B5  BV1  BV2  BV3  BV4  BV5   price
0  2282.0  2280.0  2273.0  2268.0  2265.6    1    1    1    2    3  2285.4
1  2280.6  2280.0  2278.0  2273.0  2268.0    1    1    3    1    2  2281.3
2  2282.8  2282.0  2281.4  2280.6  2280.0    1    1    1    1    1  2283.9,        B1      B2      B3      B4      B5  BV1  BV2  BV3  BV4  BV5   price
3  2281.4  2280.6  2280.0  2274.8  2273.0    1    1    1    3    1  2283.4
4  2280.6  2280.0  2279.0  2273.0  2272.2    1    1    1    1    3  2281.3
5  2278.8  2278.4  2273.0  2272.2  2269.2    1    1    1    3    1  2279.9,        B1      B2      B3      B4      B5  BV1  BV2  BV3  BV4  BV5   price
6  2279.2  2278.8  2278.4  2273.0  2272.2    1    1    1    1    3  2281.4
7  2280.0  2279.2  2278.8  2278.4  2273.0    1    1    1    1    1  2280.5
8  2280.8  2280.0  2279.2  2278.8  2278.4    1    1    1    1    1  2282.0]
[       B1      B2      B3      B4      B5  BV1  BV2  BV3  BV4  BV5   price
0  2761.0  2760.2  2

KeyboardInterrupt: 

In [98]:
read(path='data/order_books/2011/IF1101.csv', no_obs=10, slice_size=3)

[       S1      S2      S3      S4      S5  SV1  SV2  SV3  SV4  SV5   price
 0  3172.0  3173.0  3173.8  3174.6  3174.8    5    1    3    2    1  3171.9
 1  3172.8  3173.8  3174.8  3175.0  3175.8    1    1    1    2    2  3172.5
 2  3170.0  3171.2  3173.6  3174.4  3174.6    3    1    1   17    1  3169.9,
        S1      S2      S3      S4      S5  SV1  SV2  SV3  SV4  SV5   price
 3  3174.4  3174.6  3175.0  3175.8  3176.0   15    1    3    2    6  3173.8
 4  3174.4  3174.6  3175.0  3175.8  3176.0   11    2    3    2    6  3173.6
 5  3172.8  3173.4  3174.4  3174.6  3175.0   13    4    4    2    3  3172.4,
        S1      S2      S3      S4      S5  SV1  SV2  SV3  SV4  SV5   price
 6  3172.8  3173.4  3174.2  3174.4  3174.6   13    4    4    4    2  3172.6
 7  3172.2  3172.8  3173.2  3173.4  3174.2    3    3    3    5    4  3172.1
 8  3173.2  3173.4  3174.2  3174.4  3174.6    3    5    4    4    2  3172.9]